# Step 12 Character Voiceover Video

In [1]:
%pip install ffmpeg-python
!ffmpeg -version


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspe

In [2]:
from model import Story
import settings

story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

In [3]:
import ffmpeg
import math
import os

def create_looped_video_with_voiceover(video_path, audio_path, output_path):
    # Get video and audio durations
    video_info = ffmpeg.probe(video_path)
    audio_info = ffmpeg.probe(audio_path)
    video_duration = float(video_info['streams'][0]['duration'])
    audio_duration = float(audio_info['streams'][0]['duration'])
    
    # Calculate the number of loops required and create looped segments
    loops_needed = math.ceil(audio_duration / video_duration)
    segments = []
    for i in range(loops_needed):
        # Apply reverse filter on every alternate segment
        segment = ffmpeg.input(video_path)
        if i % 2 == 1:
            segment = segment.filter('reverse')
        segments.append(segment)

    # Concatenate all segments to reach or exceed the audio duration
    video = ffmpeg.concat(*segments, v=1, a=0)
    video = video.trim(end=audio_duration)  # Trim excess duration

    # Add the audio and export the final video
    final_output = ffmpeg.output(video, ffmpeg.input(audio_path), output_path, vcodec='libx264', acodec='aac')
    final_output.run(overwrite_output=True)


In [4]:
import os
from IPython.display import display, Markdown, Video

voiceover_dir = f"{settings.STORY_DIR}/step_12/voiceover"
os.makedirs(voiceover_dir, exist_ok=True)
character_videos = []
character_gifs = []

for character in story.characters:
    src_video_path = f"{settings.STORY_DIR}/step_7/characters/{character.nickname}.mp4"
    src_gif_path = f"{settings.STORY_DIR}/step_7/characters/{character.nickname}.gif"
    src_audio_path = f"{settings.STORY_DIR}/step_11/voices/{character.nickname}_catchphrase.fish.wav"
    dst_video_path = f"{voiceover_dir}/{character.nickname}.mp4"

    create_looped_video_with_voiceover(src_video_path, src_audio_path, dst_video_path)

    character_videos.append(dst_video_path)
    character_gifs.append(src_gif_path)

    # Display character name and embedded video
    display(Markdown(f"### {character.name}"))
    display(Video(dst_video_path, embed=True))


Error: ffprobe error (see stderr output for detail)

In [ ]:
import ffmpeg
import os

def combine_videos(video_paths, output_path):
    """
    Combines multiple video files into one long video, ensuring each has an audio track.

    Parameters:
    - video_paths: List of paths to video files to combine.
    - output_path: Path to save the combined output video.
    """
    
    def add_silent_audio(video_path):
        # Create a temporary file with a silent audio track
        temp_output = f"{video_path}_with_audio.mp4"
        ffmpeg.input(video_path).output(
            temp_output, vcodec='copy', acodec='aac', audio_bitrate='128k', anullsrc='1'
        ).run(overwrite_output=True)
        return temp_output
    
    # Ensure there is at least one video to combine
    if not video_paths:
        raise ValueError("The video_paths list is empty.")
    
    # Process each video to ensure it has an audio track
    processed_videos = []
    for video in video_paths:
        probe = ffmpeg.probe(video)
        has_audio = any(stream['codec_type'] == 'audio' for stream in probe['streams'])
        if not has_audio:
            # Add silent audio if the video has no audio track
            video_with_audio = add_silent_audio(video)
            processed_videos.append(video_with_audio)
        else:
            processed_videos.append(video)
    
    # Create an input stream for each processed video
    inputs = [ffmpeg.input(video) for video in processed_videos]
    
    # Concatenate videos in sequence
    combined_video = ffmpeg.concat(*[i.video for i in inputs], v=1, a=0)
    audio_concat = ffmpeg.concat(*[i.audio for i in inputs], v=0, a=1)
    
    # Output the final concatenated video
    final_output = ffmpeg.output(combined_video, audio_concat, output_path, vcodec='libx264', acodec='aac')
    final_output.run(overwrite_output=True)
    
    # Clean up any temporary files created
    for temp_file in processed_videos:
        if temp_file.endswith("_with_audio.mp4") and os.path.exists(temp_file):
            os.remove(temp_file)


In [ ]:
from PIL import Image, ImageSequence

def combine_gifs(gif_paths, output_path):
    """
    Combines multiple GIF files into one long GIF.

    Parameters:
    - gif_paths: List of paths to GIF files to combine.
    - output_path: Path to save the combined output GIF.
    """
    # Open all GIFs and extract frames
    frames = []
    for gif_path in gif_paths:
        gif = Image.open(gif_path)
        frames.extend([frame.copy() for frame in ImageSequence.Iterator(gif)])

    # Save frames as a single GIF
    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0, duration=frames[0].info['duration'])

In [ ]:
# Show the combined video

from IPython.display import display, Markdown, Video, IPImage

gif_output_path = f"{settings.STORY_DIR}/step_12/combined_video.gif"
combine_gifs(character_videos, gif_output_path)
display(IPImage(filename=gif_output_path))

output_path = f"{settings.STORY_DIR}/step_12/combined_video.mp4"
combine_videos(character_videos, output_path)
display(Video(output_path))